## Implementação Computacional
Agora, vamos ver como podemos automatizar a execução do algortimo através de programação. Nesse caso, iremos utilizar o _Python_. Entretanto, é necessário primeiro importar sua biblioteca _Numpy_ para podermos trabalhar com matrizes.

In [1]:
import numpy as np
import numpy.linalg as la

Isto feito, podemos criar as funções que serão utilizadas. Inicialmente, mostraremos o funcionamento em uma matriz gerada automaticamente. Eventualmente, estaremos utilizando redes retiradas da Internet. 
### Gerando uma matriz

In [2]:
def generate_internet(n) :
    c = np.full([n,n], np.arange(n))
    c = (abs(np.random.standard_cauchy([n,n])/2) > (np.abs(c - c.T) + 1)) 
    c = (c+1e-10) / np.sum((c+1e-10), axis=0)
    return c

Assim, criamos uma matriz quadrada do tamanho que quisermos na qual as colunas somam 1. Portanto, é uma candidata funcional para a aplicação dos métodos de _PageRank_ que visualizamos. Vamos criar uma variável _matriz_ para guardar o valor da Internet que criamos.

In [3]:
matriz = generate_internet(4)
matriz

array([[2.5e-01, 1.0e-10, 5.0e-01, 5.0e-11],
       [2.5e-01, 1.0e-10, 5.0e-11, 5.0e-11],
       [2.5e-01, 1.0e+00, 5.0e-11, 5.0e-01],
       [2.5e-01, 1.0e-10, 5.0e-01, 5.0e-01]])

Agora, o primeiro método que foi explicado: vamos calcular todos os autovalores e autovetores presentes na matriz criada. Depois, organizamos os autovalores em ordem decrescente e utilizamos esta mesma ordem para os autovetores. Podemos, portanto, remover o primeiro autovetor dessa lista: será o autovetor correspondente ao maior autovelor, e portanto, o _PageRank_ da matriz.
### Utilizando os algoritmos

In [4]:
def eigPageRank(linkMatrix):
    eVals, eVecs = la.eig(linkMatrix) # Calcula os autovalores e autovetores
    order = np.absolute(eVals).argsort()[::-1] # Ordena pelos autovalores
    eVals = eVals[order]
    eVecs = eVecs[:,order]

    r = eVecs[:, 0] # r é o principal autovetor
    return 100 * np.real(r / np.sum(r)) # Faz o vetor somar um (e multiplica por 100).

Utilizando essa função em nossa matriz, temos:

In [5]:
eigPageRank(matriz)

array([21.05263158,  5.2631579 , 31.57894737, 42.10526315])

Contudo, como vimos, podemos aproximar também através de iterações. Iniciamos com um vetor $r$, que tem todas as entradas iguais e a soma é 1. Realizamos a multiplicação da _matriz_ por $r$ e atualizamos o valor de $r$ até que a norma da diferença entre duas iterações seja menor que um número extremamente pequeno.

In [6]:
# Método por dot product
def dotPageRank(M) :
    n = M.shape[0]
    r = 100 * np.ones(n) / n # Vetor (n linhas 1/n × 100 cada)
    last = r
    r = M @ r
    while la.norm(last - r) > 0.01 :
        last = r
        r = M @ r
    return r

Se aplicarmos esse método em nossa matriz, temos:

In [7]:
dotPageRank(matriz)

array([21.05171085,  5.26457429, 31.57846332, 42.10525154])

Perceba que os resultados são bem semelhantes. Porém, ainda temos como aprimorar esse processo, como vimos anteriormente. Introduzimos então o fator de amortecimento $d$, para que o sistema seja eficiente em situações de drenagens ou ciclos. Usualmente, o fator de amortecimento utilizado é de 85%.

In [8]:
# Método por dot product (com solução para páginas que somam 0)
def dotPageRankD(linkMatrix, d) :
    n = linkMatrix.shape[0]
    M = d * linkMatrix + (1-d)/n * np.ones([n, n])
    r = 100 * np.ones(n) / n # Vetor (n linhas 1/n × 100 cada)
    last = r
    r = M @ r
    while la.norm(last - r) > 0.01 :
        last = r
        r = M @ r
    return r

Por fim, testaremos esse novo processo em nossa matriz:

In [9]:
dotPageRankD(matriz, 0.85)

array([21.87046479,  8.39676658, 31.69856198, 38.03420665])

Devido à aleatoriedade, vemos que os valores são alterados significantemente. Contudo, o importante, ou seja, o ranqueamento do _PageRank_ dos 4 sites, segue equivalente aos outros métodos.